In [2]:
from QHyper.solvers import solver_from_config
from reports.report import ExecutionReport, Division, Subworkflow, Solution, WorkflowSolution
import os

In [19]:
import sys
sys.setrecursionlimit(10000)

In [2]:
solver_config = {
    "problem": {
        "type": "workflow_scheduling",
        "encoding": "one-hot",
        "tasks_file": "workflows_data/workflows/1000genome_902_tasks.json", 
        "machines_file": "workflows_data/machines/cyfronet.json",
        "deadline": 5000,
    },
    "solver": {
        "type": "gurobi",
    }
}

gurobi = solver_from_config(solver_config)

In [3]:
# After changes in objective_function
solution_gurobi = gurobi.solve(params_inits = {"name": "wsp"})

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


Consider calling update less frequently.

Optimize a model with 8910 rows, 4510 columns and 123090 nonzeros
Model fingerprint: 0x9583e66e
Variable types: 0 continuous, 4510 integer (4510 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 1673167.3210
Presolve removed 8579 rows and 4465 columns
Presolve time: 0.16s
Presolved: 331 rows, 45 columns, 1035 nonzeros
Found heuristic solution: objective 1613742.0661
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: cutoff, 34 iterations, 0.00 seconds (0.00 work uni

In [21]:
solution_gurobi = gurobi.solve(params_inits = {"name": "wsp"})

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Consider calling update less frequently.

Optimize a model with 8910 rows, 4510 columns and 123090 nonzeros
Model fingerprint: 0x9583e66e
Variable types: 0 continuous, 4510 integer (4510 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 1673167.3210
Presolve removed 8579 rows and 4465 columns
Presolve time: 0.14s
Presolved: 331 rows, 45 columns, 1035 nonzeros
Found heuristic solution: objective 1613742.0661
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: cutoff, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [16]:
solution = gurobi.problem.decode_solution(solution_gurobi)
cost = gurobi.problem.calculate_solution_cost(solution)
time = gurobi.problem.calculate_solution_timespan(solution)
print(f"Solution: {solution}")
print(f"Cost of running full workflow: {cost}")
print(f"Time of running full workflow: {time}")

Solution: {'individuals_ID0000001': 'PrometeusGpu', 'individuals_ID0000002': 'PrometeusGpu', 'individuals_ID0000003': 'PrometeusGpu', 'individuals_ID0000004': 'PrometeusGpu', 'individuals_ID0000005': 'PrometeusGpu', 'individuals_ID0000006': 'PrometeusGpu', 'individuals_ID0000007': 'PrometeusGpu', 'individuals_ID0000008': 'PrometeusGpu', 'individuals_ID0000009': 'PrometeusGpu', 'individuals_ID0000010': 'PrometeusGpu', 'individuals_ID0000011': 'PrometeusGpu', 'individuals_ID0000012': 'PrometeusGpu', 'individuals_ID0000013': 'PrometeusGpu', 'individuals_ID0000014': 'PrometeusGpu', 'individuals_ID0000015': 'PrometeusGpu', 'individuals_ID0000016': 'PrometeusGpu', 'individuals_ID0000017': 'PrometeusGpu', 'individuals_ID0000018': 'PrometeusGpu', 'individuals_ID0000019': 'PrometeusGpu', 'individuals_ID0000020': 'PrometeusGpu', 'individuals_ID0000021': 'PrometeusGpu', 'individuals_ID0000022': 'PrometeusGpu', 'individuals_ID0000023': 'PrometeusGpu', 'individuals_ID0000024': 'PrometeusGpu', 'indi

In [20]:
def get_solver(tasks_file: str, machines_file: str, deadline: int):
    solver_config = {
        "problem": {
            "type": "workflow_scheduling",
            "encoding": "one-hot",
            "tasks_file": tasks_file, 
            "machines_file": machines_file,
            "deadline": deadline,
        },
        "solver": {
            "type": "gurobi",
        }
    }

    return solver_from_config(solver_config)

def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"

def run_tests(test_data, save_dir):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        gurobi = get_solver(tasks_file, machines_file, deadline)
        solution_gurobi = gurobi.solve(params_inits = {"name": "wsp"})
        solution = gurobi.problem.decode_solution(solution_gurobi)
        cost = gurobi.problem.calculate_solution_cost(solution)
        time = gurobi.problem.calculate_solution_timespan(solution)
        workflow = gurobi.problem.workflow
        without_division_report = ExecutionReport(
            workflow=tasks_file,
            machines=machines_file,
            deadline=deadline,
            division=Division(
                method="NoDivision",
                workflows=[
                    Subworkflow(id=0,deadline=workflow.deadline,workflow=workflow)
                ]
            ),
            solution=Solution(
                solver="Gurobi",
                workflows=[
                    WorkflowSolution(id=0,machine_assignment=solution,cost=cost,time=time)
                ],
                cost=cost,
                time=time
            )
        )
        without_division_report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [21]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json", 
    "workflows_data/workflows/1000genome_156_tasks.json", 
    "workflows_data/workflows/1000genome_492_tasks.json", 
    "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json"
]

deadlines = [
    5000,
    5000,
    5000,
    5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [22]:
run_tests(test_data, "reports/hurbol")

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 110 columns and 410 nonzeros
Model fingerprint: 0x7d058f9d
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [4e-01, 6e+06]
  Objective range  [3e-01, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 226960.38625
Presolve removed 42 rows and 110 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 226101 226960 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.261011572408e+05, best bound 2.261

In [3]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from QHyper.solvers.gurobi import Gurobi

In [10]:
import sys
print(sys.getrecursionlimit())

3000


In [2]:
tasks_file = "workflows_data/workflows/srasearch_22_tasks.json"
machines_file = "workflows_data/machines/cyfronet.json"
deadline = 5000
workflow_genome = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingOneHot(workflow_genome)

# gurobi = Gurobi(problem=wsp)
# solution_gurobi = gurobi.solve(params_inits = {"name": "wsp"})

KeyboardInterrupt: 

In [1]:
from QHyper.problems.algorithms.utils import wfworkflow_to_qhyper_workflow
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm, preprocess_workflow

In [17]:
def divide_into_solvers(tasks_file: str, machines_file: str, deadline: int, n_parts: int):
    workflow = Workflow(tasks_file, machines_file, deadline)
    workflow = wfworkflow_to_qhyper_workflow(preprocess_workflow(workflow), machines_file, deadline)
    subworkflows, deadlines = HeftBasedAlgorithm().decompose(workflow, n_parts)
    deadlines = list(map(lambda d: round(d), deadlines))
    problems = map(lambda w, d: WorkflowSchedulingOneHot(wfworkflow_to_qhyper_workflow(w, machines_file, d)), subworkflows, deadlines)
    return list(map(lambda p: Gurobi(problem=p), problems))

def append_solution(solver, solution, idx, report):
    cost = solver.problem.calculate_solution_cost(solution)
    time = solver.problem.calculate_solution_timespan(solution)
    workflow = solver.problem.workflow
    report.division.workflows.append(Subworkflow(id=idx,deadline=workflow.deadline,workflow=workflow))
    report.solution.workflows.append(WorkflowSolution(id=idx,machine_assignment=solution,cost=cost,time=time))
    report.solution.cost += cost
    report.solution.time += time

def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"

def run_tests_division(test_data, save_dir, n_parts):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        solvers = divide_into_solvers(tasks_file, machines_file, deadline, n_parts)
        solutions = list(map(lambda x: x[1].solve(params_inits = {"name": f"wsp{x[0]}"}), enumerate(solvers)))
        solutions_decoded = list(map(lambda p, s: p.problem.decode_solution(s), solvers, solutions))
        
        report = ExecutionReport(
            workflow=tasks_file,
            machines=machines_file,
            deadline=deadline,
            division=Division(
                method="HeftBasedAlgorithm",
                workflows=[]
            ),
            solution=Solution(
                solver="Gurobi",
                workflows=[],
                cost=0,
                time=0
            )
        )
        
        for idx, (solver, solution) in enumerate(zip(solvers, solutions_decoded)):
            append_solution(solver, solution, idx, report)

        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [20]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json", 
    "workflows_data/workflows/1000genome_156_tasks.json", 
    "workflows_data/workflows/1000genome_492_tasks.json", 
    "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json",
    "workflows_data/machines/cyfronet.json"
]

deadlines = [
    5000,
    5000,
    5000,
    5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [21]:
run_tests_division(test_data, "reports/my", 2)

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 65 columns and 120 nonzeros
Model fingerprint: 0x9c6ea067
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+06]
  Objective range  [2e+01, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 209611.41938
Presolve removed 24 rows and 65 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 209575 209611 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.095750033650e+05, best bound 2.0957500